In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import logging


In [26]:
# Read in and look at the data
df = pd.read_csv('stock_trans_data.csv')

df.head()

,symbol,historical_dates,open,close,purchase_date,purchase_price,sell_date,sell_price,actual_return,days_to_sell,take_profit_price,stop_out_price,hit_take_profit,sector
0,CRM,"[datetime.date(2018, 12, 3), datetime.date(201...","[146.3, 142.73, 134.8, 141.9, 135.16, 140.52, ...","[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...",2023-11-20,223.20,2023-11-30,249.48,315.37,10.0,231.57,220.9680,1,Technology
1,NET,"[datetime.date(2019, 9, 13), datetime.date(201...","[18.1, 18.6, 18.45, 18.83, 20, 18.9, 20.1, 21....","[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...",2023-11-20,73.37,2023-12-14,83.38,340.34,24.0,81.62,72.6363,1,Technology
2,RCL,"[datetime.date(2018, 11, 23), datetime.date(20...","[107.1, 111.04, 110.53, 111.38, 113.05, 111.2,...","[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...",2023-11-20,105.26,2023-11-21,102.96,-57.50,1.0,107.73,104.2074,0,Consumer Cyclical
3,SPOT,"[datetime.date(2018, 12, 6), datetime.date(201...","[131.01, 136.89, 134.2, 133.81, 130.3, 129.78,...","[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...",2023-11-20,178.86,2023-12-04,193.85,224.84,14.0,187.08,177.0714,1,Communication Services
4,FERG,"[datetime.date(2021, 3, 8), datetime.date(2021...","[120.51, 123, 124, 124, 123.72, 125.48, 126.66...","[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...",2023-11-20,164.78,2023-11-20,164.63,-2.41,0.0,165.06,163.1322,0,Industrials


In [27]:
# Convert date to a datetime object and then create seperate columns for day, month, year, day as Monday, Friday ....
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_day'] = df['purchase_date'].dt.day
df['purchase_month'] = df['purchase_date'].dt.month
df['purchase_year'] = df['purchase_date'].dt.year
df['sell_date'] = pd.to_datetime(df['sell_date'])
df['sell_day'] = df['sell_date'].dt.day
df['sell_month'] = df['sell_date'].dt.month
df['sell_year'] = df['sell_date'].dt.year




# Function to calculate SMA5
def calculate_sma5(close_prices):
    sma5 = []
    for i in range(len(close_prices)):
        if i < 4:  # Predict the SMA5 for the first four dates
            sma5.append(close_prices[i])
        else:
            sma5.append(np.mean(close_prices[i-4:i+1]))
    return sma5

# Apply the function to the 'close' column
df['SMA5'] = df['close'].apply(lambda x: calculate_sma5(eval(x)))

# Inspect the result
# df[['close', 'SMA5']].head()
df

,symbol,historical_dates,open,close,purchase_date,purchase_price,sell_date,sell_price,actual_return,days_to_sell,...,stop_out_price,hit_take_profit,sector,purchase_day,purchase_month,purchase_year,sell_day,sell_month,sell_year,SMA5
0,CRM,"[datetime.date(2018, 12, 3), datetime.date(201...","[146.3, 142.73, 134.8, 141.9, 135.16, 140.52, ...","[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...",2023-11-20,223.20,2023-11-30,249.48,315.37,10.0,...,220.9680,1,Technology,20,11,2023,30.0,11.0,2023.0,"[144.15, 139.24, 142.06, 135.95, 139.733999999..."
1,NET,"[datetime.date(2019, 9, 13), datetime.date(201...","[18.1, 18.6, 18.45, 18.83, 20, 18.9, 20.1, 21....","[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...",2023-11-20,73.37,2023-12-14,83.38,340.34,24.0,...,72.6363,1,Technology,20,11,2023,14.0,12.0,2023.0,"[18, 18.63, 18.75, 19.59, 18.744, 19.118000000..."
2,RCL,"[datetime.date(2018, 11, 23), datetime.date(20...","[107.1, 111.04, 110.53, 111.38, 113.05, 111.2,...","[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...",2023-11-20,105.26,2023-11-21,102.96,-57.50,1.0,...,104.2074,0,Consumer Cyclical,20,11,2023,21.0,11.0,2023.0,"[110.28, 111.46, 110.74, 112.54, 111.330000000..."
3,SPOT,"[datetime.date(2018, 12, 6), datetime.date(201...","[131.01, 136.89, 134.2, 133.81, 130.3, 129.78,...","[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...",2023-11-20,178.86,2023-12-04,193.85,224.84,14.0,...,177.0714,1,Communication Services,20,11,2023,4.0,12.0,2023.0,"[136.83, 135.31, 130.79, 128.37, 131.954, 129...."
4,FERG,"[datetime.date(2021, 3, 8), datetime.date(2021...","[120.51, 123, 124, 124, 123.72, 125.48, 126.66...","[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...",2023-11-20,164.78,2023-11-20,164.63,-2.41,0.0,...,163.1322,0,Industrials,20,11,2023,20.0,11.0,2023.0,"[119.49, 122.35, 120.6, 125, 122.4159999999999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,WELL,"[datetime.date(2019, 8, 26), datetime.date(201...","[88.16, 89.2, 89.47, 89.36, 89.84, 89.38, 92.1...","[88.6, 88.96, 88.89, 89.68, 89.56, 91.38, 92.4...",2024-08-12,115.96,2024-08-23,119.44,167.03,11.0,...,114.8004,1,Real Estate,12,8,2024,23.0,8.0,2024.0,"[88.6, 88.96, 88.89, 89.68, 89.138, 89.6939999..."
297,K,"[datetime.date(2019, 8, 16), datetime.date(201...","[63.89, 63.83, 64.26, 63.54, 63.42, 63.89, 63....","[63.78, 64.43, 63.23, 63.35, 63.87, 63.25, 63....",2024-08-12,74.29,2024-08-14,80.09,435.00,2.0,...,73.5471,1,Consumer Defensive,12,8,2024,14.0,8.0,2024.0,"[63.78, 64.43, 63.23, 63.35, 63.73199999999999..."
298,VIRT,"[datetime.date(2019, 8, 26), datetime.date(201...","[18.26, 17.71, 17.77, 18.01, 18.24, 18.89, 18....","[18, 17.76, 17.99, 18.34, 18.8, 18.89, 18.58, ...",2024-08-15,30.31,NaT,NaN,NaN,NaN,...,30.0069,0,Financial Services,15,8,2024,NaN,NaN,NaN,"[18, 17.76, 17.99, 18.34, 18.178, 18.356, 18.5..."
299,GGAL,"[datetime.date(2019, 8, 26), datetime.date(201...","[14.89, 13.7, 12.54, 12.3, 11.97, 11.76, 9.97,...","[13.95, 12.87, 12.74, 11.9, 10.84, 9.57, 11.05...",2024-08-15,35.89,2024-08-22,33.50,-2.40,7.0,...,35.5311,0,Financial Services,15,8,2024,22.0,8.0,2024.0,"[13.95, 12.87, 12.74, 11.9, 12.459999999999999..."


In [28]:
def calculate_sma20(close_prices):
    sma20 = []
    for i in range(len(close_prices)):
        if i < 19:  # Not enough data for SMA5
            sma20.append(close_prices[i])
        else:
            sma20.append(np.mean(close_prices[i-19:i+1]))
    return sma20

# Apply the function to the 'close' column
df['SMA20'] = df['close'].apply(lambda x: calculate_sma20(eval(x)))

# Inspect the result
df[['close', 'SMA20']].head()

,close,SMA20
0,"[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...","[144.15, 139.24, 142.06, 135.95, 137.27, 136.7..."
1,"[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...","[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,..."
2,"[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...","[110.28, 111.46, 110.74, 112.54, 111.63, 113.0..."
3,"[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...","[136.83, 135.31, 130.79, 128.37, 128.47, 125.5..."
4,"[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...","[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1..."


In [29]:
# Function to calculate SMA5 slope based on the previous 4 days
def calculate_sma5_slope_with_prediction(sma5_list):
    sma5_slope = []
    for i in range(len(sma5_list)):
        if i < 4:  # Predict the slope for the first four entries
            if i > 0:  # Calculate the slope between the first two available points
                predicted_slope = round((sma5_list[i] - sma5_list[i-1]) , 2)
            else:
                predicted_slope = 0  # If no prior data, assume flat slope
            sma5_slope.append(predicted_slope)
        else:
            # Slope = (SMA5[today] - SMA5[4 days ago]) / 4
            slope = round((sma5_list[i] - sma5_list[i-4]) / 4, 2)
            sma5_slope.append(slope)
    return sma5_slope

# Apply the function to the 'close' column
df['SMA5_Slope'] = df['SMA5'].apply(lambda x: calculate_sma5_slope_with_prediction(x))

# Inspect the result
df[['SMA5', 'SMA5_Slope']].head()

,SMA5,SMA5_Slope
0,"[144.15, 139.24, 142.06, 135.95, 139.733999999...","[0, -4.91, 2.82, -6.11, -1.1, -0.24, -0.91, 0...."
1,"[18, 18.63, 18.75, 19.59, 18.744, 19.118000000...","[0, 0.63, 0.12, 0.84, 0.19, 0.12, 0.21, 0.1, 0..."
2,"[110.28, 111.46, 110.74, 112.54, 111.330000000...","[0, 1.18, -0.72, 1.8, 0.26, 0.11, 0.43, -0.1, ..."
3,"[136.83, 135.31, 130.79, 128.37, 131.954, 129....","[0, -1.52, -4.52, -2.42, -1.22, -1.4, -0.75, -..."
4,"[119.49, 122.35, 120.6, 125, 122.4159999999999...","[0, 2.86, -1.75, 4.4, 0.73, 0.38, 1.01, 0.1, 0..."


In [30]:
# Function to calculate SMA5 slope based on the previous 4 days
def calculate_sma20_slope_with_prediction(sma20_list):
    sma20_slope = []
    for i in range(len(sma20_list)):
        if i < 19:  # Predict the slope for the first four entries
            if i > 0:  # Calculate the slope between the first two available points
                predicted_slope = round((sma20_list[i] - sma20_list[i-1]) , 2)
            else:
                predicted_slope = 0  # If no prior data, assume flat slope
            sma20_slope.append(predicted_slope)
        else:
            slope = round((sma20_list[i] - sma20_list[i-4]) / 19, 2)
            sma20_slope.append(slope)
    return sma20_slope

# Apply the function to the 'close' column
df['SMA20_Slope'] = df['SMA20'].apply(lambda x: calculate_sma20_slope_with_prediction(x))

# Inspect the result
df[['SMA20', 'SMA20_Slope']].head()

,SMA20,SMA20_Slope
0,"[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...","[0, -4.91, 2.82, -6.11, 1.32, -0.48, 3.3, 1.05..."
1,"[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...","[0, 0.63, 0.12, 0.84, -0.84, 1.12, 1.09, -0.25..."
2,"[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...","[0, 1.18, -0.72, 1.8, -0.91, 1.44, 1.18, -5.12..."
3,"[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...","[0, -1.52, -4.52, -2.42, 0.1, -2.92, 0.13, -5...."
4,"[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...","[0, 2.86, -1.75, 4.4, -0.36, 2.02, -0.33, -1.8..."


In [31]:
from statistics import mode

# Create a new list of 1s and 0s. 1 = increase from one sma to the next days sma, 0 = decrease from on sma to the next days sma
def calculate_increases(lst):
    probabilities = []
    for i in range(len(lst) - 1):
        if lst[i] < lst[i + 1]:
            probabilities.append(1)
        else:
            probabilities.append(0)
    probabilities.append(0)
    most_common = mode(probabilities)
    return most_common

# create new SMA<value>_prob columns and SMA<value>_slope_prob columns based on the result of the calculate increase function.
def get_increase_probability_and_create_new_column(initial_column, created_column):
    probs = []
    for x in df[initial_column]:
        prob = calculate_increases(x)
        probs.append(prob)
    df[created_column] = probs

# create a list of column names to run this on
old_columns = ['SMA5', 'SMA20', 'SMA5_Slope', 'SMA20_Slope']
for name in old_columns:
    get_increase_probability_and_create_new_column(name, name+'_prob')
    
# Calculate summary statistics for 'open' and 'close' prices
df['open_mean'] = df['open'].apply(lambda x: np.mean(eval(x)))
df['open_std'] = df['open'].apply(lambda x: np.std(eval(x)))
df['close_mean'] = df['close'].apply(lambda x: np.mean(eval(x)))
df['close_std'] = df['close'].apply(lambda x: np.std(eval(x)))
df['SMA5_last'] = df['SMA5'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA20_last'] = df['SMA20'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA5_Slope_last'] = df['SMA5_Slope'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA20_Slope_last'] = df['SMA20_Slope'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)


# drop all columns that contains lists as their data
df = df.drop(['SMA5','SMA20','SMA5_Slope','SMA20_Slope','open','close','historical_dates', 'actual_return'], axis=1)
# inspect the dataframe in its current state
df

,symbol,purchase_date,purchase_price,sell_date,sell_price,days_to_sell,take_profit_price,stop_out_price,hit_take_profit,sector,...,SMA5_Slope_prob,SMA20_Slope_prob,open_mean,open_std,close_mean,close_std,SMA5_last,SMA20_last,SMA5_Slope_last,SMA20_Slope_last
0,CRM,2023-11-20,223.20,2023-11-30,249.48,10.0,231.57,220.9680,1,Technology,...,0,0,194.874448,41.238154,194.878719,41.079058,231.268,219.90650,1.87,0.35
1,NET,2023-11-20,73.37,2023-12-14,83.38,24.0,81.62,72.6363,1,Technology,...,0,0,66.832950,38.665226,66.932882,38.640908,79.714,76.48300,0.49,0.14
2,RCL,2023-11-20,105.26,2023-11-21,102.96,1.0,107.73,104.2074,0,Consumer Cyclical,...,0,0,81.513144,25.881074,81.473604,25.901222,104.394,93.66850,1.59,0.25
3,SPOT,2023-11-20,178.86,2023-12-04,193.85,14.0,187.08,177.0714,1,Communication Services,...,1,0,174.833222,67.292948,174.899467,67.200348,184.978,177.20050,0.88,0.19
4,FERG,2023-11-20,164.78,2023-11-20,164.63,0.0,165.06,163.1322,0,Industrials,...,1,0,138.396361,17.321011,138.696428,17.230391,165.182,157.66450,1.37,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,WELL,2024-08-12,115.96,2024-08-23,119.44,11.0,119.41,114.8004,1,Real Estate,...,1,0,78.268211,14.744209,78.246701,14.753526,117.098,115.16450,0.15,0.09
297,K,2024-08-12,74.29,2024-08-14,80.09,2.0,76.50,73.5471,1,Consumer Defensive,...,0,0,64.184265,5.475001,64.174996,5.475372,75.600,64.91800,1.06,0.20
298,VIRT,2024-08-15,30.31,NaT,NaN,NaN,31.17,30.0069,0,Financial Services,...,0,0,23.084998,4.877328,23.075294,4.873304,29.814,28.72100,0.06,0.02
299,GGAL,2024-08-15,35.89,2024-08-22,33.50,7.0,36.89,35.5311,0,Financial Services,...,0,0,12.707613,6.600854,12.708771,6.619103,34.311,31.41975,-0.16,0.06


In [32]:
from sklearn.preprocessing import LabelEncoder
# Initialize the Label Encoder
label_encoder = LabelEncoder()
# Fit and transform the 'symbol' column
df['symbol_encoded'] = label_encoder.fit_transform(df['symbol'])
# Fit and transform the 'purchase_date' column
df['purchase_date_encoded'] = label_encoder.fit_transform(df['purchase_date'])
# Fit and transform the 'sell_date' column
df['sell_date_encoded'] = label_encoder.fit_transform(df['sell_date'])

# inspect the dataframe in its current state
df


,symbol,purchase_date,purchase_price,sell_date,sell_price,days_to_sell,take_profit_price,stop_out_price,hit_take_profit,sector,...,open_std,close_mean,close_std,SMA5_last,SMA20_last,SMA5_Slope_last,SMA20_Slope_last,symbol_encoded,purchase_date_encoded,sell_date_encoded
0,CRM,2023-11-20,223.20,2023-11-30,249.48,10.0,231.57,220.9680,1,Technology,...,41.238154,194.878719,41.079058,231.268,219.90650,1.87,0.35,50,0,7
1,NET,2023-11-20,73.37,2023-12-14,83.38,24.0,81.62,72.6363,1,Technology,...,38.665226,66.932882,38.640908,79.714,76.48300,0.49,0.14,106,0,16
2,RCL,2023-11-20,105.26,2023-11-21,102.96,1.0,107.73,104.2074,0,Consumer Cyclical,...,25.881074,81.473604,25.901222,104.394,93.66850,1.59,0.25,129,0,1
3,SPOT,2023-11-20,178.86,2023-12-04,193.85,14.0,187.08,177.0714,1,Communication Services,...,67.292948,174.899467,67.200348,184.978,177.20050,0.88,0.19,148,0,9
4,FERG,2023-11-20,164.78,2023-11-20,164.63,0.0,165.06,163.1322,0,Industrials,...,17.321011,138.696428,17.230391,165.182,157.66450,1.37,0.11,69,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,WELL,2024-08-12,115.96,2024-08-23,119.44,11.0,119.41,114.8004,1,Real Estate,...,14.744209,78.246701,14.753526,117.098,115.16450,0.15,0.09,179,88,106
297,K,2024-08-12,74.29,2024-08-14,80.09,2.0,76.50,73.5471,1,Consumer Defensive,...,5.475001,64.174996,5.475372,75.600,64.91800,1.06,0.20,94,88,104
298,VIRT,2024-08-15,30.31,NaT,NaN,NaN,31.17,30.0069,0,Financial Services,...,4.877328,23.075294,4.873304,29.814,28.72100,0.06,0.02,170,89,107
299,GGAL,2024-08-15,35.89,2024-08-22,33.50,7.0,36.89,35.5311,0,Financial Services,...,6.600854,12.708771,6.619103,34.311,31.41975,-0.16,0.06,76,89,105


In [33]:
# Create a seperate dataframe for the rows that are still showing as the position being open. This will be used for prediction
future_df = df[(df['sell_price'].isna())]
future_df = future_df.copy()

# drop the open positions from the training and test data
df = df.dropna(subset=['sell_price'])


# Now you have the last 5 open and close prices as features for your model create model used for testing future data
X = df.drop(['hit_take_profit', 'symbol', 'purchase_date', 'sell_date'], axis=1)
y = df['hit_take_profit']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

logging.info(f"Accuracy: {accuracy}")
logging.info("Classification Report:\n", classification_rep)



ValueError: could not convert string to float: 'Consumer Cyclical'

In [ ]:
X_future = future_df.drop(['hit_take_profit', 'symbol', 'purchase_date', 'sell_date'], axis=1)
future_predictions = rf_model.predict(X_future)

In [ ]:
future_df['hit_take_profit_predicted'] = future_predictions

In [ ]:
future_df.to_csv('future_predictions.csv', index=False)

In [ ]:
future_probabilities = rf_model.predict_proba(X_future)
future_probabilities

In [ ]:
logging.info(df['hit_take_profit'].value_counts())


In [ ]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rf_model, X, y, cv=5)
logging.info("Cross-validation scores:", cv_scores)
logging.info("Mean cross-validation score:", np.mean(cv_scores))

In [ ]:
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
logging.info(feature_importance_df.sort_values(by='importance', ascending=False))
